In [1]:
import json
import glob
import pandas as pd
import kiwi

In [3]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
    return data

path = glob.glob('dataset/*.json')[:10000]
df = []
for p in path:
	df.append(load_data(p))
df = pd.DataFrame(df)
df = df[(df['종교']==1)&(df['정치성향Y/N']!='Y')]
df = df[['문장', '대상하이라이트']]
df

,문장,대상하이라이트
117,대명 천지에 이렇게 ''미개한''' 종교가 있나?,대명 천지에 이렇게 미개한 ''종교'''가 있나?
253,#@이름#의 이름으로 포교활동가는 개신교인들 없냐? 폭력에 굴하지 않는 #@이름#...,''#@이름#'''의 이름으로 포교활동가는 ''개신교'''인들 없냐? 폭력에 굴하...
255,#@이름# ''견찰'''이 815에 광화문에서 했던건데 탈레반도 하네 ㅎㅎ,#@이름# ''견찰'''이 815에 광화문에서 했던건데 ''탈레반'''도 하네 ㅎㅎ
303,근데 예맨애들 제주도에 들어와서 그대로 있냐? 이슬람 애들 늘어서 좋을거 하나 없는데,근데 ''예맨애'''들 제주도에 들어와서 그대로 있냐? ''이슬람 애'''들 늘어서...
319,탈레반은 말을 거꾸로하는 희한한 습관이 있네 반대파를 용서하고 복수는 없다 두려워말...,''탈레반'''은 말을 거꾸로하는 희한한 습관이 있네 반대파를 용서하고 복수는 없다...
...,...,...
9796,어떤''새끼'''가 희열을 느낀다.미얀하다.고맙다라고 했던가! 그''새끼'''가 탈레반,어떤새끼가 희열을 느낀다.미얀하다.고맙다라고 했던가! 그새끼가 탈레반
9812,''개톨릭'''의 민낯이 드러났다. 사단의 무리들이 아니고서야 어찌 어린 영혼을 말...,''개톨릭'''의 민낯이 드러났다. 사단의 무리들이 아니고서야 어찌 어린 영혼을 말...
9845,한국 기독교 부동산소유 밝히면 재밌을듯,''한국 기독교''' 부동산소유 밝히면 재밌을듯
9918,이제 중국바이러스 없어지길 기대하는건 바보같은 생각. 온갖 변이가 판치는 판국에 백...,이제 중국바이러스 없어지길 기대하는건 바보같은 생각. 온갖 변이가 판치는 판국에 백...


In [4]:
# 특수 문자 제거
df.loc[:, '문장'] = df['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]", "", regex=True)

# 빈 문자열 제거
df = df[df['문장'].str.strip() != '']

len(df)

102

In [5]:
# 형태소 분석 및 명사 추출
nouns = []  #빈 리스트 nouns를 생성합니다. 여기에는 텍스트에서 추출된 명사들이 저장

for sentence in df['문장']:   #df['문장']는 데이터프레임 df의 '문장' 열에 있는 각 문장을 순차적으로 가져옵니다
    tokens = kiwi.analyze(sentence)  #kiwi.analyze(sentence)는 Kiwi 형태소 분석기를 이용해 각 문장을 형태소 분석하여 그 결과를 tokens 변수에 저장
    for token in tokens[0][0]:   #tokens[0][0]을 통해 분석된 첫 번째 문장의 첫 번째 분석 결과를 순회하면서 각 형태소를 확인
        if token[1] == 'NNG' or token[1] == 'NNP':  # 일반 명사와 고유 명사인 경우에만 해당 형태소를 다음 단계에서 처리
            nouns.append(token[0])  #명사로 판별된 형태소(단어)를 nouns 리스트에 추가
nouns

AttributeError: module 'kiwi' has no attribute 'analyze'

In [ ]:
# 한글자 단어 제거하기 (예:내, 저, 것 등등)
filtered_nouns = [word for word in nouns if len(word) >= 2]

In [ ]:
# 불용어 제거
stopwords = set(['엣팁'])
filtered_nouns = [word for word in filtered_nouns if word not in stopwords]

In [ ]:
# 단어 빈도수 계산
word_counts = Counter(filtered_nouns)

In [ ]:
# 시각화 - 그래프
common_words = word_counts.most_common(10)
words, counts = zip(*common_words)

plt.rc('font', family='NanumBarunGothic')
plt.figure(figsize=(10, 6))
colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
plt.bar(words, counts, color=colors)

plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('Top 10 Words')
plt.savefig('my_bar.png', format='png')
plt.show()

In [ ]:
# 워드 클라우드 생성
wordcloud = WordCloud(font_path='BMDOHYEON_ttf.ttf',
                      width=800, height=400,
                      background_color='white', mask = mask_img).generate_from_frequencies(word_counts)

In [ ]:
# 시각화 - 워드 클라우드
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Wordcloud')
plt.savefig('my_wordcloud.png', format='png')
plt.show()